# Generative AI Challenge: Character Consistency

Name: Daan Quaadvliet 

Number: 231146

## 1. Solution Overview

beautiful scenery nature glass bottle landscape, , purple galaxy bottle,

[WRITE HERE A SHORT EXPLANATION ABOUT YOUR IDEA TO SOLVE THE CHARACTER CONSISTENCY PROBLEM]

## 2. Load Story

We start by loading a story. The code below loads a random story from the server and stores the scene descritions in the ``story_scenes`` list. 

Every time you run this code, you will get a different story. Your solution must work with any story given by the server.

In [ ]:
import requests

response = requests.get("https://edirlei.com/buas/get-story.php")
story_scenes = response.json()

for i in range(len(story_scenes)):
    print("SCENE " + str(i + 1) + ": " + story_scenes[i])

## 3. Preprocess Scene Descriptions

[EXPLAIN HERE ALL PREPROCESSING STEPS PERFORMED IN THE SCENE DESCRIPTIONS]

[ADD MORE MARKDOWN OR CODE CELLS IF NEEDED]

In [ ]:
# [WRITE HERE ALL THE CODE TO PREPROCESS THE SCENE DESCRIPTIONS]

##### PLACEHOLDER CODE JUST TO COPY THE ORIGINAL SCENE DESCRIPTIONS - REPLACE IT WITH YOUR CODE #####
processed_story_scenes = []
for scene in story_scenes:
    processed_story_scenes.append(scene) 
#####################################################################################################

## 4. Image Generation with Improved Character Consistency

[EXPLAIN HERE HOW YOUR METHOD TO IMPROVE CHARACTER CONSISTENCY IS IMPLEMENTED]

[EXPLAIN YOUR SOLUTION STEP-BY-STEP]

[IF YOUR SOLUTION INVOLVES EXTERNAL STEPS THAT ARE NOT IMPLEMENTED IN THIS NOTEBOOK (SUCH AS FINE-TUNING A MODEL), EXPLAIN THE PROCESS HERE]

[ADD MORE MARKDOWN OR CODE CELLS IF NEEDED]

In [ ]:
# [WRITE HERE THE CODE TO GENERATE THE IMAGES]

##### PLACEHOLDER CODE TO GENERATE IMAGES USING COMFYUI - REPLACE IT WITH YOUR CODE #####

import uuid
import json
import urllib
import random
import io
import websocket
import os
import base64
from IPython.display import Image, display

comfyui_server_address = "buas1.edirlei.com"
comfyui_juggernaut_xl_workflow_path = "./workflow_juggernaut_xl.json"
comfyui_client_id = str(uuid.uuid4())
comfyui_username = "adsai"
comfyui_password = "uuUP4whjX29cF3cxwrX3SY5Mm9TmtASkdgpaNCTHZ9"
comfyui_credentials = base64.b64encode(f"{comfyui_username}:{comfyui_password}".encode('utf-8')).decode('utf-8')
comfyui_auth_header = f"Basic {comfyui_credentials}"

def comfyui_queue_prompt(prompt):
    p = {"prompt": prompt, "client_id": comfyui_client_id}
    data = json.dumps(p).encode('utf-8')
    req = urllib.request.Request(f"http://{comfyui_server_address}/prompt", data=data, headers = {"Authorization": comfyui_auth_header})
    return json.loads(urllib.request.urlopen(req).read())

def comfyui_get_image(filename, subfolder, folder_type):
    data = {"filename": filename, "subfolder": subfolder, "type": folder_type}
    url_values = urllib.parse.urlencode(data)
    req = urllib.request.Request(f"http://{comfyui_server_address}/view?{url_values}", headers = {"Authorization": comfyui_auth_header})
    with urllib.request.urlopen(req) as response:
        return response.read()

def comfyui_get_history(prompt_id):
    req = urllib.request.Request(f"http://{comfyui_server_address}/history/{prompt_id}", headers = {"Authorization": comfyui_auth_header})
    with urllib.request.urlopen(req) as response:
        return json.loads(response.read())

def comfyui_get_images(ws, prompt):
    prompt_id = comfyui_queue_prompt(prompt)['prompt_id']
    output_images = {}
    while True:
        out = ws.recv()
        if isinstance(out, str):
            message = json.loads(out)
            if message['type'] == 'executing':
                data = message['data']
                if data['node'] is None and data['prompt_id'] == prompt_id:
                    break 
        else:
            continue

    history = comfyui_get_history(prompt_id)[prompt_id]
    for o in history['outputs']:
        for node_id in history['outputs']:
            node_output = history['outputs'][node_id]
            if 'images' in node_output:
                images_output = []
                for image in node_output['images']:
                    image_data = comfyui_get_image(image['filename'], image['subfolder'], image['type'])
                    images_output.append(image_data)
            output_images[node_id] = images_output
    return output_images

def comfyui_generate_image_juggernaut_xl(prompt, negative_prompt, seed, output_filename):
    with open(comfyui_juggernaut_xl_workflow_path, "r") as file:
        workflowdata = json.load(file)
    if seed == -1:
        seed = random.randint(0, 2**32 - 1)
    workflowdata["6"]["inputs"]["text"] = prompt
    workflowdata["7"]["inputs"]["text"] = negative_prompt    
    workflowdata["3"]["inputs"]["seed"] = seed
    ws = websocket.WebSocket()
    ws.connect("ws://{}/ws?clientId={}".format(comfyui_server_address, comfyui_client_id), header={"Authorization": comfyui_auth_header})
    images = comfyui_get_images(ws, workflowdata)
    if len(images) > 0:
        bytestream = io.BytesIO(images["9"][0])
        with open(output_filename, "wb") as outfile:
            outfile.write(bytestream.getbuffer())

rnd_story_id = random.randint(1, 9999)
os.makedirs(".//images", exist_ok=True)
for i in range(len(processed_story_scenes)):
    comfyui_generate_image_juggernaut_xl(processed_story_scenes[i], "", -1, ".//images//story_" + str(rnd_story_id) + "_scene_" + str(i + 1) + ".png")
for i in range(len(processed_story_scenes)):
    display(Image(filename=".//images//story_" + str(rnd_story_id) + "_scene_" + str(i + 1) + ".png", width=400))
    print(processed_story_scenes[i])

#########################################################################################

## 5. Results and Discussion

[EXPLAIN HERE THE RESULTS: WHAT WORKS WELL? WHAT DOES NOT WORK SO WELL?]

## 6. Conclusions

[WRITE HERE YOUR MAIN CONCLUSIONS ABOUT THE SOLUTION: ARE YOU HAPPY WITH THE RESULTS?]

[WHAT WERE THE MAIN CHALLENGES OF DEVELOPING THIS SOLUTION?]

[WHAT ARE THE LIMITATIONS OF YOUR SOLUTION?]

[WHAT ARE THE FUTURE STEPS THAT YOU THINK COULD BE EXPLORED TO IMPROVE THE RESULTS]